In [ ]:
pip install control

In [ ]:
import control

In [ ]:
s = control.tf('s')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Inverse Response System

In [ ]:
K1 = 5.0
T1 = 5.0

In [ ]:
P1 = K1/(T1*s+1)

In [ ]:
K2 = 2.0
T2 = 1.0

In [ ]:
P2 = K2/(T2*s+1)

In [ ]:
Process = control.parallel(P1, -P2)

In [ ]:
t, y = control.step_response(Process)

In [ ]:
# Plot step response
plt.figure(1)
plt.plot(t, y)
plt.xlabel('Time (s)')
plt.ylabel('Output')
plt.title('System Step Response')
plt.grid()
plt.show()

## Export Data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
t = np.transpose(t)
y = np.transpose(y)

In [ ]:
df = pd.DataFrame({'t':t, 'y':y})

In [ ]:
df.to_excel("RespostaInversa.xlsx", index=False)

# Parameter Tuning

In [ ]:
df = pd.read_excel('RespostaInversa.xlsx')

In [ ]:
df.describe()

In [ ]:
Max_t = df['t'].max()

In [ ]:
Delta_t = df._get_value(1, 't') - df._get_value(0, 't')

In [ ]:
Count_t = len(df['t'])

## Optimize

In [ ]:
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error

In [ ]:
def fun(x):
    global df

    K1 = x[0]
    T1 = x[1]

    K2 = x[2]
    T2 = x[3]

    P1 = K1/(T1*s+1)
    P2 = K2/(T2*s+1)
    sys = control.parallel(P1, -P2)

    # Simulate step response
    t_fun, y_fun = control.step_response(sys, T=Max_t, T_num=Count_t)
    df_fun = pd.DataFrame({'t_fun':t_fun, 'y_fun':y_fun})

    mse = mean_squared_error(df_fun['y_fun'], df['y'])
    return mse

In [ ]:
x0 = [1, 1, 1, 1]

In [ ]:
bnds = ((0, None), (0, None), (0, None), (0, None))

In [ ]:
res = minimize(fun, x0, method='Powell', bounds=bnds, tol=1e-15)

In [ ]:
K1_opt = res.x[0]
T1_opt = res.x[1]
K2_opt = res.x[2]
T2_opt = res.x[3]

P1_opt = K1_opt/(T1_opt*s+1)
P2_opt = K2_opt/(T2_opt*s+1)
sys_opt = control.parallel(P1_opt, -P2_opt)

In [ ]:
print('K1=', K1_opt)
print('T1=', T1_opt)
print('K2=', K2_opt)
print('T2=', T2_opt)

## Plots

In [ ]:
t_opt, y_opt = control.step_response(sys_opt, T=Max_t, T_num=Count_t)

In [ ]:
# Plot step response
plt.plot(t_opt,y_opt)
plt.scatter(df['t'], df['y'], marker= ".", color = "red")
plt.xlabel('Time (s)')
plt.ylabel('Output')
plt.title('System Step Response')
plt.grid()
plt.show()